In [1]:
import time
from typing import Dict

import cv2
import numpy as np

from pystream import Pipeline, Stage

In [2]:
# Cycle period for the input data
INPUT_PERIOD = 0.2
# Time to run the pipeline
ON_TIME = 5

In [3]:
class DummyStage(Stage):
    """A dummy stage that performs some convolutions to the input 2D array,
    and count how many input it has processed.
    
    It is recommended to define 'name' property in the stage instance init
    """

    def __init__(self, name: str) -> None:
        """Initialize stage

        Args:
            name (str): stage name
        """        
        self.count = 0
        self.name = name
        self.kernel = np.random.randint(-10, 10, size=(5, 5))

    def __call__(self, data: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
        """Main data processing of the stage, a mandatory method of the Stage 
        class. Note that the stage does not return a new dict object, but only
        modifies the input object and returns it as the output

        Args:
            data (Dict[str, np.ndarray]): the input array, packed
                in a dictionary

        Returns:
            Dict[str, np.ndarray]: the output array, packed
            in a dictionary
        """
        img = data["data"]
        for _ in range(100):
            img = cv2.filter2D(src=img, ddepth=-1, kernel=self.kernel)
        data["data"] = img
        self.count += 1
        return data

    def cleanup(self):
        """Cleanup method, called at the end of the pipeline.
        in this example, we want to reset the counter to 0"""
        self.count = 0

In [4]:
def generate_data() -> Dict[str, np.ndarray]:
    img = np.random.randint(0, 255, size=(480, 720, 3), dtype=np.uint8)
    return {"data": img}

def create_pipeline() -> Pipeline:
    # First, create the pipeline instance, we want to use the profiler
    pipeline = Pipeline(input_generator=generate_data, use_profiler=True)
    # Now, add 5 dummy stages to the pipeline.
    pipeline.add(DummyStage("Stage1"))
    pipeline.add(DummyStage("Stage2"))
    pipeline.add(DummyStage("Stage3"))
    pipeline.add(DummyStage("Stage4"))
    pipeline.add(DummyStage("Stage5"))
    return pipeline

In [5]:
pipeline = create_pipeline()
print("Starting pipeline in serial...")
pipeline.serialize()
pipeline.start_loop(INPUT_PERIOD)
time.sleep(ON_TIME)
print("Stopping pipeline...")
pipeline.stop_loop()

# Let's try read the last result and do cleanup
latest = pipeline.get_results()
print("Last output shape:")
print(latest["data"])
pipeline.cleanup()

# Get the profile
profile = pipeline.get_profiles()
print("Pipeline profiling results:")
print(profile)

Exception in thread PyStream-Automation:
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "d:\AdibFile\AdibProgram\pystream-pipeline\pystream\pipeline\automation.py", line 45, in _loop_handler
    self.pipeline.forward(data)
  File "d:\AdibFile\AdibProgram\pystream-pipeline\pystream\pipeline\pipeline.py", line 126, in forward
    return self.pipeline.forward(pipeline_data)
  File "d:\AdibFile\AdibProgram\pystream-pipeline\pystream\pipeline\serial_pipeline.py", line 37, in forward
    data = stage(data)
  File "d:\AdibFile\AdibProgram\pystream-pipeline\pystream\stage\container.py", line 47, in __call__
    data.data = self.stage(data.data)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_21020\1850571075.py", line 31, in __ca

Starting pipeline in serial...
PipelineData(data=PipelineData(data={'data': array([[[ 53, 201,  16],
        [139,  90, 204],
        [231, 244,  34],
        ...,
        [205, 228,  81],
        [167, 211, 173],
        [103, 183,  16]],

       [[ 51,  79,  13],
        [  0, 130, 188],
        [113, 147,  49],
        ...,
        [222, 253,  32],
        [140, 107, 202],
        [149, 200, 199]],

       [[168,   7,  31],
        [ 89,  56, 170],
        [  0, 143, 177],
        ...,
        [ 31, 198,  11],
        [241,  35,  31],
        [140,  78, 203]],

       ...,

       [[115, 184, 134],
        [152, 249, 246],
        [206,  78,  68],
        ...,
        [177, 244, 130],
        [249,  45,  83],
        [ 38, 118,  50]],

       [[131,  51, 128],
        [ 84, 207, 208],
        [242, 164, 249],
        ...,
        [  0, 138, 112],
        [159, 150,  43],
        [127, 182, 131]],

       [[ 76, 251, 150],
        [237,  16,  12],
        [ 33,  84, 186],
        ...

TypeError: 'NoneType' object is not subscriptable